# Map

This notebook is dedicated to the data processing required by the map visualisation. 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
BASE_DIR = '../climate-change-earth-surface-temperature-data/'

## Extracting data for a month

We first extract data for a given month in order to facilitate early stages of the map design.

In [15]:
# Load data
FILENAME = 'GlobalLandTemperaturesByCity.csv'
df = pd.read_csv(BASE_DIR + FILENAME)

We extract data for a given month:

In [16]:
# Pick a month
DATE = '1900-01-01'
df = df[df.dt == DATE].reset_index(drop=True)
df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1900-01-01,-0.989,0.588,Århus,Denmark,57.05N,10.33E
1,1900-01-01,5.763,1.008,Çorlu,Turkey,40.99N,27.69E
2,1900-01-01,0.947,0.771,Çorum,Turkey,40.99N,34.08E
3,1900-01-01,-25.819,1.058,Öskemen,Kazakhstan,50.63N,82.39E
4,1900-01-01,-20.970,1.505,Ürümqi,China,44.20N,87.20E


Coordinates need to be converted to the regular format:

In [14]:
def map_coordinate(coo):
    """
    Correct coordinate format
    """
    if coo[-1] == 'S' or coo[-1] == 'W':
        coo = '-' + coo
    
    return coo[:-1]

In [20]:
df.Latitude = df.Latitude.map(map_coordinate)
df.Longitude = df.Longitude.map(map_coordinate)

Finally, we export the `json` data:

In [24]:
df = df.drop(['dt', 'AverageTemperatureUncertainty'], axis=1)

In [34]:
records = [{DATE: df.to_dict(orient='records')}]

In [38]:
import json

OUTNAME = 'temp_city_{}_.json'.format(DATE)
OUTPATH = '../data/'
with open(OUTPATH + OUTNAME, 'w') as f:
    json.dump(records, f, ensure_ascii=False)